In [18]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot

using StaticArrays: SVector
using LinearAlgebra: normalize
using RigidBodyDynamics.Contact: HalfSpace3D
using GeometryTypes: HyperRectangle, Vec
using CoordinateTransformations: LinearMap
using Colors: RGB
using Rotations: RotY, Quat, rotation_between
using MeshCat: setobject!, settransform!, MeshPhongMaterial

In [ ]:
mechanism = StrandbeestRobot.mechanism(add_flat_ground=false)
ground_normal = normalize(SVector(1., 0, 5))
add_environment_primitive!(mechanism, 
    HalfSpace3D(Point3D(root_frame(mechanism), 0., 0, 0),
        FreeVector3D(root_frame(mechanism), ground_normal)))

gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()))

In [25]:
#NBSKIP
open(gui)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 51, 52, 54, 55, 57, 58, 60, 61, 63, 64, 66, 67, 69, 70, 72, 73, 75, 76, 78, 79]
└ @ Optim /home/rdeits/.julia/packages/Optim/fabGe/src/multivariate/solvers/constrained/fminbox.jl:212


In [ ]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)

floatingjoint = findjoint(mechanism, "floating_base")
rot = Quat(rotation_between(SVector(0., 0, 1), ground_normal))
set_configuration!(state, floatingjoint, [rot.w, rot.x, rot.y, rot.z, 0, 0, 0.9])

setobject!(gui.visualizer.visualizer[:ground], HyperRectangle(Vec(-50, -2, -0.01), Vec(100, 4, 0.02)), 
MeshPhongMaterial(color=RGB(0.5, 0.5, 0.5)))
settransform!(gui.visualizer.visualizer[:ground], LinearMap(rot))

copyto!(gui.visualizer, state)

In [31]:
# Simulate the robot
damping = function (τ, t, state)
    τ .= -0.1 .* velocity(state)
end
dynamics = Dynamics(mechanism, damping)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)

 39.752277 seconds (4.45 M allocations: 985.935 MiB, 1.12% gc time)
